   mkdir bgee_coexpression  
   cd bgee_coexpression  
   wget ftp://ftp.bgee.org/current/download/calls/expr_calls/Homo_sapiens_expr_simple.tsv.zip  
   unzip Homo_sapiens_expr_simple.tsv.zip 

In [4]:
! ls -1

BGee_coexpression_profile.ipynb
FA_1_core_complex.txt
FA_2_effector_proteins.txt
FA_3_associated_proteins.txt
Homo_sapiens_expr_simple.tsv
Homo_sapiens_expr_simple.tsv.zip
Untitled1.ipynb


: 1

In [2]:
head -1 Homo_sapiens_expr_simple.tsv | tr '\t' '\n' | grep -n .

1:Gene ID
2:"Gene name"
3:Anatomical entity ID
4:"Anatomical entity name"
5:Expression
6:Call quality
7:Expression rank


In [5]:
# how many rows
wc -l < Homo_sapiens_expr_simple.tsv

8884623


In [7]:
# how many gene identifiers?
cut -f1 Homo_sapiens_expr_simple.tsv | sort -u | wc -l

59172


In [26]:
# how many gene symbols?
cut -f2 Homo_sapiens_expr_simple.tsv | sort -u | wc -l

57344


That is alot of gene ids & names,  
many more than _could_ have informative nomenclature

In [9]:
# how many tissues?
cut -f3 Homo_sapiens_expr_simple.tsv | sort -u | wc -l

312


If every gene is checked for in every tissue  we will see  
59172 * 312 == 18,461,664  tissue-gene measurements

but we only see 8,884,623  rows   
so less than half gene-tissue combinations possible could be reported. 

In [19]:
# tissue is ontology terms (yea!).
cut -f3 Homo_sapiens_expr_simple.tsv | cut -f1 -d ':' |sort |uniq -c | sort -nr 

8649326 UBERON
 235296 CL
      1 Anatomical entity ID


This makes for very interesting "future directions" where gene expression profiles
can be scored at various granularities based on anatomical windows, 
(if it turns out to be feasable)   
there are 312 tissue terms represented, 

In [ ]:
cut -f3 Homo_sapiens_expr_simple.tsv | sort | uniq -c | sort -nr| awk '{print $1 "\t" $2}'  > tissue_count.tab

In [ ]:
# gnuplot  -p -e 'plot ""tissue_count.tab" using 1'

how much coverage do they get?  

![tissue_count](tissue_count.png)  

that the first ~ hundred @60000 are well covered and can be left alone,   
the later parts could possibly use some ontological binning,  
if it leads to fewer tissues recorded for more genes.  


In [11]:
# what is measured
cut -f5 Homo_sapiens_expr_simple.tsv | sort | uniq -c | sort -n

      1 Expression
2893316 absent
5991306 present


existance.  (but only among the ~half that are measured),  
two thirds are there and one third is not there

each gene-tissue combination has three states
 - unreported   
 - absent
 - present
 
combining these we can have six combinations 

 - `unreported unreported`
 - `unreported absent`
 - `unreported present`
 - `absent absent`
 - `absent present`
 - `present present`
 
anything and `unreported` should remain `unreported`  
any that agree should strengthen   
any that disagree should weaken  

The first scheme that comes to mind is to let:   
    absent     = -1  
    unreported =  0  
    present    =  1  
then multiply each gene tissue comparison and sum the resulting vector.

note: 
This has the a similar but not as bad effect as my previous scoreing scheme
in [human_protein_atlas_coexpression](https://github.com/TomConlin/human_protein_atlas_coexpression)  
there the best match score approached that of unreported. 
here the 50/50 [agree/disagree] match score aproaches unreported.  

Which at least has a chance of not mattering if significance   
reqires more agreement than disagreement, which seems likely.

In [14]:
# we also have a three tier quality meteric
cut -f6 Homo_sapiens_expr_simple.tsv | sort | uniq -c | sort -n

      1 Call quality
 124978 bronze quality
2921827 gold quality
5837817 silver quality


but this is not imediately useful

also have a rank.

```
Rank scores of expression calls are normalized across genes, conditions and species. Low score means that the gene is highly expressed in the condition. Max rank score in all species: 4.10e4. Min rank score varies across species.
```
41,000

In [16]:
cut -f7 Homo_sapiens_expr_simple.tsv | head

Expression rank
9.77e3
33.3
2.96e4
9.96e3
1.71e4
3.31e4
2.60e4
1.86e4
4.06e4
cut: write error: Broken pipe


That is far more nuanced than yes|no|maybe

Check up front if our Fanconi genes are represented

In [16]:
cut -f2 Homo_sapiens_expr_simple.tsv | tr -d '"' |sort -u > bgee_gene_symbol.txt

In [9]:
wc -l < bgee_gene_symbol.txt

57344


In [11]:
head FA_1_core_complex.txt

NCBIGene:2175	FANCA
NCBIGene:2187	FANCB
NCBIGene:2176	FANCC
NCBIGene:2178	FANCE
NCBIGene:2188	FANCF
NCBIGene:2189	FANCG
NCBIGene:55120	FANCL
NCBIGene:57697	FANCM
NCBIGene:2177	FANCD2
NCBIGene:55215	FANCI


In [14]:
cut -f2 FA_*.txt | sort > target_gene_symbol.txt

In [18]:
comm -12 bgee_gene_symbol.txt target_gene_symbol.txt | wc -l

25


Great! that is 25 out of 27 just checking symbols   
the other two may show up via identifiers   
if any nomenclature is out of date.
